In [26]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from ipywidgets import interactive, FloatSlider
%matplotlib inline

In [31]:
F = lambda k, x0: (lambda x: -k * (x - x0))
T = lambda m: (lambda v: 0.5 * m * v**2)
V = lambda k, x0: (lambda x: 0.5 * k * (x - x0)**2)
E = lambda k, x0, m: (lambda x, v: T(m)(v) + V(k, x0)(x))

# Verlet step functions
update_position = lambda dt, m: lambda x, v, f: x + v*dt + 0.5*f/m*dt**2
update_velocity = lambda dt, m: lambda v, f_old, f_new: v + 0.5*dt/m*(f_old + f_new)

In [32]:
# Verlet integration function
def verlet_integration(x0, v0, m, k, x_eq, dt, num_steps):
    f = F(k, x_eq)
    energy = E(k, x_eq, m)

    x, v = x0, v0
    f_old = f(x)

    trajectory = [(x, v, energy(x, v))]

    for _ in range(num_steps):
        x = update_position(dt, m)(x, v, f_old)
        f_new = f(x)
        v = update_velocity(dt, m)(v, f_old, f_new)
        f_old = f_new
        e = energy(x, v)

        trajectory.append((x, v, e))

    return np.array(trajectory)

In [33]:
def analyze_energy_conservation(m, k, x0, v0, x_eq=0, dt_values=None, num_periods=10):
    if dt_values is None:
        dt_values = np.logspace(-3, -1, 10)

    omega = np.sqrt(k/m)
    T = 2*np.pi/omega
    num_steps = int(num_periods * T / dt_values.min())

    std_energies = []

    for dt in dt_values:
        trajectory = verlet_integration(x0, v0, m, k, x_eq, dt, num_steps)
        energies = [step[2] for step in trajectory]  # Energy is the 3rd element
        std_energies.append(np.std(energies))

    return dt_values, std_energies

def plot_results(m, k, x0, v0):
    dt_values, std_energies = analyze_energy_conservation(m, k, x0, v0)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    fig.suptitle('Energy Conservation Analysis')

    ax1.loglog(dt_values, std_energies, 'bo-')
    ax1.set_xlabel('Time Step (dt)')
    ax1.set_ylabel('Standard Deviation of Energy')
    ax1.set_title('Energy Fluctuations vs Time Step')
    ax1.grid(True)

    # Fit a line to log-log data
    log_dt = np.log(dt_values)
    log_std_e = np.log(std_energies)
    coeffs = np.polyfit(log_dt, log_std_e, 1)
    ax1.plot(dt_values, np.exp(coeffs[1]) * dt_values**coeffs[0], 'r--',
             label=f'Fit: slope = {coeffs[0]:.2f}')
    ax1.legend()

    trajectory = verlet_integration(x0, v0, m, k, 0, dt_values[5], 1000)
    time = np.arange(len(trajectory)) * dt_values[5]
    positions = [step[0] for step in trajectory]
    velocities = [step[1] for step in trajectory]
    energies = [step[2] for step in trajectory]
    kinetic_energies = [0.5 * m * v**2 for v in velocities]
    potential_energies = [0.5 * k * x**2 for x in positions]

    ax2.plot(time, kinetic_energies, label='Kinetic Energy')
    ax2.plot(time, potential_energies, label='Potential Energy')
    ax2.plot(time, energies, label='Total Energy')
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Energy')
    ax2.set_title('Energy Components over Time')
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    plt.show()

    print(f"The fitted slope is {coeffs[0]:.2f}, compared to the expected value of 2 for O(dt²) scaling.")

interactive_widget = interactive(plot_results,
                                 m=FloatSlider(min=0.1, max=5.0, step=0.1, value=1.0, description='Mass (m)'),
                                 k=FloatSlider(min=0.1, max=10.0, step=0.1, value=1.0, description='Spring Constant (k)'),
                                 x0=FloatSlider(min=-2.0, max=2.0, step=0.1, value=1.0, description='Initial Position (x0)'),
                                 v0=FloatSlider(min=-2.0, max=2.0, step=0.1, value=0.0, description='Initial Velocity (v0)'))

display(interactive_widget)

interactive(children=(FloatSlider(value=1.0, description='Mass (m)', max=5.0, min=0.1), FloatSlider(value=1.0,…